### OBJECTIVE:
The objective of this project is to predict the sale price of houses using various features such as house size, number of rooms, location, year built, and other property characteristics.
By building a predictive model, we aim to estimate SalePrice for any given house in the dataset.

In [ ]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from scipy.stats import shapiro

In [ ]:
# Step 2: Load Dataset
df = pd.read_csv(r"C:\Users\dell\OneDrive\Documents\train.csv")  
df.head()

In [ ]:
df.shape


In [ ]:
target = df.SalePrice
print(target)


In [ ]:
#seperate numerical and categorical features
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(numerical_features)
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
print(categorical_features)

In [ ]:
#basic statistics of numerical and categorical features

df[numerical_features].describe()

In [ ]:
df[categorical_features].describe()

# step 3: Target variable -sales price analysis

In [ ]:

df['SalePrice'].describe()


In [ ]:
%matplotlib inline


In [ ]:
#distribution plots of sales price
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.histplot(df['SalePrice'], kde=True)
plt.title("Distribution of SalePrice")
plt.xlabel("SalePrice")
plt.ylabel("Frequency")
plt.show()


In [ ]:
df['SalePrice'].skew()


In [ ]:
#outlier detection using boxplot
plt.figure(figsize=(8,4))
sns.boxplot(x=df['SalePrice'])
plt.title("Boxplot of SalePrice")
plt.show()


In [ ]:
#log transformation of sales price
import numpy as np

df['SalePrice_log'] = np.log1p(df['SalePrice'])


In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df['SalePrice_log'], kde=True)
plt.title("Distribution of Log-Transformed SalePrice")
plt.xlabel("log(SalePrice)")
plt.ylabel("Frequency")
plt.show()


In [ ]:
#compare skewness before vs after
print("Original Skewness:", df['SalePrice'].skew())
print("Log Transformed Skewness:", df['SalePrice_log'].skew())


In [ ]:
!pip install ydata-profiling


In [ ]:
!pip install ipywidgets



In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="EDA Report", explorative=True)
profile


In [ ]:
profile.to_file("EDA_Report.html")


In [ ]:
import os
os.getcwd()


In [ ]:
profile.to_file(
    r"C:\Users\dell\Desktop\_EDA_Report.html"
)


4️⃣ Automated EDA Insights (YData Profiling)

### 🔹 Missing Values
The automated profiling report revealed that several features contain a very high proportion of missing values.  
- **Alley (93.8%)**, **PoolQC (99.5%)**, **Fence (80.8%)**, and **MiscFeature (96.3%)** have excessive missing data.  
These features provide limited information and may introduce noise, therefore they will be considered for removal in subsequent steps.

Moderate missing values were observed in:
- **LotFrontage (17.7%)**
- **MasVnrType** and **MasVnrArea (~60%)**  
These features will be imputed rather than removed.

---

### 🔹 Correlation & Redundant Features
The profiling report identified several highly correlated feature pairs, indicating potential multicollinearity:
- GarageArea ↔ GarageCars  
- GrLivArea ↔ 2ndFlrSF  
- 1stFlrSF ↔ TotalBsmtSF  
- Exterior1st ↔ Exterior2nd  

To reduce multicollinearity, one feature from each highly correlated pair will be retained based on interpretability and predictive relevance.

---

### 🔹 Skewness & Zero Inflation
Some numerical features showed extreme skewness or a large proportion of zero values:
- **Highly skewed:** MiscVal  
- **Mostly zero-valued:** PoolArea, 3SsnPorch, LowQualFinSF, ScreenPorch  

Such features are likely to have limited impact on house prices and will be evaluated carefully before inclusion.

---

### 🔹 Categorical Imbalance
Several categorical features were found to be highly imbalanced:
- Utilities, Street, Condition2, RoofMatl  

Highly imbalanced variables often provide limited information gain and may not contribute meaningfully to prediction.

---

### 🔹 Identifier Columns
The feature **Id** was found to be unique for each record and uniformly distributed.  
Since it does not carry predictive information, it will be removed from the dataset.

---

### 🔹 Summary
Automated EDA using YData Profiling provided a structured overview of missing values, correlations, skewness, and imbalance in the dataset.  
These insights were used to guide early feature elimination and inform preprocessing and feature selection decisions in the following steps.

# step 5 :Manual EDA

In [ ]:

# A) sales price vs numerical variables

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

num_features = [
    'GrLivArea', 'TotalBsmtSF', '1stFlrSF',
    'OverallQual', 'GarageArea', 'YearBuilt'
]

for col in num_features:
    plt.figure(figsize=(6,4))
    sns.scatterplot(x=df[col], y=df['SalePrice'])
    plt.title(f'SalePrice vs {col}')
    plt.xlabel(col)
    plt.ylabel('SalePrice')
    plt.show()


### SalePrice vs GrLivArea
- Strong positive relationship observed
- Larger living area generally increases SalePrice
- Few extreme outliers present which may affect linear models


In [ ]:
#B) sales price vs categorical plots
cat_features = [
    'OverallQual', 'Neighborhood',
    'HouseStyle', 'KitchenQual',
    'GarageFinish'
]

for col in cat_features:
    plt.figure(figsize=(8,4))
    sns.boxplot(x=df[col], y=df['SalePrice'])
    plt.xticks(rotation=45)
    plt.title(f'SalePrice vs {col}')
    plt.show()


### SalePrice vs OverallQual
- Median SalePrice increases sharply with better quality
- Strong monotonic relationship
- Indicates OverallQual is one of the most important predictors


In [ ]:
#C) Correlation with sales price
corr = df.corr(numeric_only=True)['SalePrice'].sort_values(ascending=False)
corr


In [ ]:
plt.figure(figsize=(6,8))
corr[1:15].plot(kind='barh')
plt.title('Top Numerical Correlations with SalePrice')
plt.show()


### Correlation Analysis
- GrLivArea, OverallQual,garagecars,garage area and TotalBsmtSF show strong correlation with SalePrice
- Some features are highly correlated with each other, indicating multicollinearity
- Weakly correlated features may be excluded from modeling


# step 6 : feature selection 


In [ ]:
#A) drop ID column
df=df.drop(columns='Id')
df.columns

In [ ]:
#B. Drop Features with Very High Missing Values (>70%)

missing_percent = (df.isnull().sum() / len(df)) * 100
missing_percent.sort_values(ascending=False)
high_missing_cols = missing_percent[missing_percent > 70].index
high_missing_cols
df = df.drop(columns=high_missing_cols)
df.shape

In [ ]:
#C. Drop Highly Imbalanced Categorical Features

for col in df.select_dtypes(include='object').columns:
    print(col)
    print(df[col].value_counts(normalize=True).head(), '\n')


In [ ]:
imbalanced_cols = [
    'Utilities',
    'Street',
    'Condition2',
    'RoofMatl',
    'Heating'
]

df = df.drop(columns=imbalanced_cols, errors='ignore')
df.shape

In [ ]:
#D. Remove Highly Correlated Features (Numerical)
corr = df.corr(numeric_only=True)
corr['SalePrice'].sort_values(ascending=False)


In [ ]:
df = df.drop(columns=[
    'GarageCars',
    'TotRmsAbvGrd',
    'Exterior2nd',
    'BsmtUnfSF',
    '1stFlrSF'
], errors='ignore')
df.shape

In [ ]:
#E. Select Final Feature Set (Optional Explicit Selection)
num_features = [
    'OverallQual',
    'GrLivArea',
    'TotalBsmtSF',
    'GarageArea',
    'YearBuilt',
    'YearRemodAdd',
    'LotArea',
    'MasVnrArea'
]

cat_features = [
    'Neighborhood',
    'MSZoning',
    'HouseStyle',
    'KitchenQual',
    'ExterQual',
    'BsmtQual',
    'GarageFinish',
    'SaleCondition'
]

final_features = num_features + cat_features + ['SalePrice']
df = df[final_features]
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

# STEP 7 : FEATURE CONSTRUCTION 

In [ ]:

# create new feature total area
df['TotalArea'] = df['GrLivArea'] + df['TotalBsmtSF'] + df['GarageArea']


In [ ]:
#create new variable house age
df['HouseAge'] = df['YearRemodAdd'] - df['YearBuilt']


In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols


# step:8  MISSING VALUE TREATMENT

In [ ]:
df.isnull().sum().sort_values(ascending=False)


# Handle categorical missing values (structural missingness)
# GarageFinish → Missing means No Garage

In [ ]:
df['GarageFinish'] = df['GarageFinish'].fillna('None')


# BsmtQual → Missing means No Basement

In [ ]:
df['BsmtQual'] = df['BsmtQual'].fillna('None')


# Handle numerical missing values (structural zero)
# MasVnrArea → Missing means No masonry veneer

In [ ]:
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)


In [ ]:
df.isnull().sum()


# Missing Value Treatment

### Missing values were handled using domain-specific logic.  
### Categorical features where missingness indicated absence (e.g., garage or basement)
### were explicitly labeled as "None".  
### For numerical features where missing values represented non-existence (e.g., masonry veneer),
### values were imputed as zero.  
### This approach preserves semantic meaning and avoids introducing artificial bias.

In [ ]:
df.shape

## 📌 Step: Train–Test Split
🎯 Objective

Separate features (X) and target (y)

Create training and testing sets

Prevent data leakage before encoding & modeling

In [ ]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = df.drop(columns=['SalePrice'])
y=df['SalePrice']



In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Check shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Step 9:Outliers Handeling

In [ ]:
#Step A: Combine X_train and y_train (temporary)
train_data = X_train.copy()
train_data['SalePrice'] = y_train
train_data.shape

In [ ]:
# Get all numerical columns including the target
num_cols = train_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(num_cols)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

num_cols = ['SalePrice', 'GrLivArea', 'TotalArea', 'LotArea', 'GarageArea','OverallQual','TotalBsmtSF','YearBuilt', 'YearRemodAdd','MasVnrArea','HouseAge' ]

# Set figure size
plt.figure(figsize=(15, 10))

# Plot boxplots
for i, col in enumerate(num_cols, 1):
    plt.subplot(len(num_cols)//3 + 1, 3, i)
    sns.boxplot(y=train_data[col])
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
#check outliers usign IQR
# Define a function to detect outliers
def detect_outliers(df, cols):
    outlier_indices = {}
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
        outlier_indices[col] = outliers
        print(f"{col} -> {len(outliers)} outliers")
    return outlier_indices

outlier_dict = detect_outliers(train_data, num_cols)

In [ ]:
#define columns to cap
cap_cols = [
    'SalePrice',
    'GrLivArea',
    'TotalArea',
    'LotArea',
    'GarageArea',
    'TotalBsmtSF',
    'MasVnrArea'
]

In [ ]:
#IQR capping function
def cap_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    df[col] = df[col].clip(lower, upper)
    return df


In [ ]:
#apply capping
for col in cap_cols:
    train_data = cap_outliers(train_data, col)


In [ ]:
X_train = train_data.drop('SalePrice', axis=1)
y_train = train_data['SalePrice']

In [ ]:
X_train

In [ ]:
#replot to verify
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15,10))
for i, col in enumerate(cap_cols, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(y=train_data[col])
    plt.title(col)
plt.tight_layout()
plt.show()


### HouseAge outliers represent valid old properties. Since they are meaningful and not errors, I retained them to preserve information and avoid bias.


# Step 10:Creating pipeline and ONE HOT ENCHODING /ORIDINAL ENCHODING FOR CATEGORICAL VARIABLES


In [ ]:

# Identify categorical columns
cat_cols = X_train.select_dtypes(include='object').columns
cat_cols


In [ ]:
for col in cat_cols:
    print(f"\n{col}")
    print(df[col].unique())


In [ ]:
for col in cat_cols:
    print(f"\n{col}")
    print(df[col].value_counts())


In [ ]:
import matplotlib.pyplot as plt

for col in cat_cols:
    df[col].value_counts().plot(kind='bar', figsize=(6,3))
    plt.title(col)
    plt.ylabel("Count")
    plt.show()


In [ ]:
#SELECT ORDINAL COLUMNS 
ordinal_cols = [
    'ExterQual',
    'KitchenQual',
    'BsmtQual',
    'GarageFinish'
]


In [ ]:
#NOMIAL COLUMNS
nominal_cols = [col for col in cat_cols if col not in ordinal_cols]


In [ ]:
ordinal_features = [
    'ExterQual',
    'KitchenQual',
    'BsmtQual',
    'GarageFinish'
]
nominal_features = [
    'Neighborhood',
    'HouseStyle',
    'MSZoning',
    'SaleCondition'
]
numerical_features = [
    'TotalArea',
    'GrLivArea',
    'OverallQual',
    'GarageArea',
    'LotArea',
    'YearBuilt',
    'YearRemodAdd',
    'HouseAge'
]


In [ ]:
#DEFINE ORINAL CATEGORICAL ORDER
from sklearn.preprocessing import OrdinalEncoder

ordinal_categories = [
    ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],   # ExterQual
    ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],   # KitchenQual
    ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],   # BsmtQual
    ['None', 'Unf', 'RFn', 'Fin']             # GarageFinish
]



In [ ]:
#PREPROCESSING PIPELINE
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline



In [ ]:
#CREATE COLUMNS TRANSFORMER
ordinal_transformer = OrdinalEncoder(
    categories=ordinal_categories,
    handle_unknown='use_encoded_value',
    unknown_value=-1
)

nominal_transformer = OneHotEncoder(
    handle_unknown='ignore',
    sparse_output=False
)

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('ord', ordinal_transformer, ordinal_features),
        ('nom', nominal_transformer, nominal_features)
    ],
    remainder='drop'
)
preprocessor


In [ ]:
#APPLY ENCODING ON TRAIN SPLIT ONLY TO PREVENT DATA LEAKAGE
# Fit on training data
# Fit on training data
X_train_encoded = preprocessor.fit_transform(X_train)

# Transform test data
X_test_encoded = preprocessor.transform(X_test)



In [ ]:
X_train_encoded.shape

In [ ]:
X_test_encoded.shape

In [ ]:
pipe.named_steps

In [ ]:
# Get feature names
feature_names = (
    numerical_features +
    ordinal_features +
    list(preprocessor.named_transformers_['nom']
         .get_feature_names_out(nominal_features))
)

print(len(feature_names))
print(feature_names)


In [ ]:
X_train_encoded

# STEP 11: LINEAR REGRESSION MODEL IMPLEMENTATION AND CHECKING ASSUMPTIONS

In [ ]:
from sklearn.linear_model import LinearRegression

mlr_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

mlr_pipe.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# ---- Basic metrics ----
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_test = mean_absolute_error(y_test, y_pred_test)

# ---- Adjusted R² ----
n = X_train.shape[0]   # number of observations
p = X_train_encoded.shape[1]  # number of features AFTER encoding

adj_r2_train = 1 - (1 - r2_train) * (n - 1) / (n - p - 1)
adj_r2_test = 1 - (1 - r2_test) * (n - 1) / (n - p - 1)

print(f"Train R²: {r2_train:.4f}")
print(f"Train Adjusted R²: {adj_r2_train:.4f}")

print(f"Test R²: {r2_test:.4f}")
print(f"Test Adjusted R²: {adj_r2_test:.4f}")

print(f"Test RMSE: {rmse_test:.2f}")
print(f"Test MAE: {mae_test:.2f}")



# 📊 Model Performance Interpretation (Multiple Linear Regression)
### 🔹 Training Performance

Train R² = 0.8854

Train Adjusted R² = 0.8796

### Interpretation:

The model explains ~88.5% of the variance in house prices on the training data.

The adjusted R² is slightly lower, which is expected because it penalizes the inclusion of many predictors.

The small difference between R² and adjusted R² indicates that most features contribute meaningful information and the model is not relying on unnecessary predictors.

### 🔹 Test Performance

Test R² = 0.8147

Test Adjusted R² = 0.8054

### Interpretation:

On unseen data, the model explains ~81.5% of the variance in house prices.

The close values of test R² and adjusted R² suggest that the model generalizes well and does not heavily overfit the training data.

The moderate drop from training R² (0.885 → 0.815) indicates slight overfitting, which is common in linear models with many encoded features, but not severe.

# 🔹 Error Metrics (Practical Interpretation)

RMSE = ₹37,699

MAE = ₹21,028

### Interpretation:

On average, the model’s prediction error is around ₹21,000 (MAE).

RMSE being higher than MAE indicates the presence of some large errors, meaning the model occasionally struggles with extreme or very expensive properties.

Since housing prices naturally have high variance, these error values are reasonable and acceptable for this dataset.    

In [ ]:
#coefficients and intercepts

lin_model = mlr_pipe.named_steps['model']
feature_names = mlr_pipe.named_steps['preprocessor'].get_feature_names_out()
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': lin_model.coef_
}).sort_values(by='Coefficient', key=abs, ascending=True)

coef_df.head(30)


In [ ]:
#intercept
print("Intercept:", lin_model.intercept_)


In [ ]:
1. Linearity

In [ ]:
y_train_pred = mlr_pipe.predict(X_train)
residuals = y_train - y_train_pred

plt.figure(figsize=(6,4))
sns.scatterplot(x=y_train_pred, y=residuals)
plt.axhline(0, color='red')
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.title("Residuals vs Predicted (Linearity Check)")
plt.show()


### “Linearity is reasonably satisfied, but variance of residuals increases with fitted values, indicating heteroscedasticity.”

In [ ]:
2. Homoschedasticity

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
import statsmodels.api as sm

X_train_sm = preprocessor.transform(X_train)
X_train_sm = sm.add_constant(X_train_sm)

bp_test = het_breuschpagan(residuals, X_train_sm)

print("Breusch-Pagan p-value:", bp_test[1])
print("Heteroscedasticity is present")

In [ ]:
3.Normality of residuals

In [ ]:
# Histogram
sns.histplot(residuals, kde=True)
plt.title("Residual Distribution")
plt.show()

# Q-Q plot
sm.qqplot(residuals, line='s')
plt.title("Q-Q Plot of Residuals")
plt.show()

# Shapiro test
from scipy.stats import shapiro
stat, p = shapiro(residuals)
print("Shapiro p-value:", p)


### normality is also voileted due to heteroschedasticity 

In [ ]:
# 4. Mullticollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_vif = preprocessor.transform(X_train)
vif_df = pd.DataFrame()
vif_df["Feature"] = feature_names
vif_df["VIF"] = [
    variance_inflation_factor(X_vif, i)
    for i in range(X_vif.shape[1])
]

vif_df.sort_values("VIF", ascending=False).head(10)


In [ ]:
#5. No auto-correlation

In [ ]:
from statsmodels.stats.stattools import durbin_watson

dw = durbin_watson(residuals)
print("Durbin-Watson:", dw)


### Applying log transformation and recheck the assumption

In [ ]:
# Log-transform target
y_log = np.log1p(y)   # log(1 + SalePrice)

# Train-test split (same X, new y)
X_train, X_test, y_train_log, y_test_log = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)


In [ ]:
mlr_log_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

mlr_log_pipe.fit(X_train, y_train_log)


In [ ]:
# Predictions (log scale)
y_train_pred_log = mlr_log_pipe.predict(X_train)
y_test_pred_log = mlr_log_pipe.predict(X_test)

# Convert back to original price scale
y_train_pred = np.expm1(y_train_pred_log)
y_test_pred = np.expm1(y_test_pred_log)

y_train_actual = np.expm1(y_train_log)
y_test_actual = np.expm1(y_test_log)


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print("Train R²:", r2_score(y_train_actual, y_train_pred))
print("Test R²:", r2_score(y_test_actual, y_test_pred))

print("Test RMSE:", np.sqrt(mean_squared_error(y_test_actual, y_test_pred)))
print("Test MAE:", mean_absolute_error(y_test_actual, y_test_pred))


In [ ]:
# recheck the assumptions

In [ ]:
1.normality

In [ ]:
residuals = y_train_log - y_train_pred_log
fitted = y_train_pred_log
from scipy.stats import shapiro
shapiro(residuals)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

sns.histplot(residuals, kde=True)
plt.title("Residuals Distribution (Log Target)")
plt.show()

stats.probplot(residuals, dist="norm", plot=plt)
plt.show()


In [ ]:
plt.scatter(fitted, residuals)
plt.axhline(0, color='red')
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Log Target)")
plt.show()


In [ ]:
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan

X_sm = sm.add_constant(
    mlr_log_pipe.named_steps['preprocessor'].transform(X_train)
)

bp_test = het_breuschpagan(residuals, X_sm)
bp_test


# 📌 STEP 12: Regularized Regression Models (Ridge & Lasso)

## Why Regularized Regression?

The Multiple Linear Regression model showed good predictive performance, but key classical assumptions 
(normality, homoscedasticity, and absence of multicollinearity) were violated.

These violations are common in real-world datasets, especially after one-hot encoding, where multicollinearity 
and high-dimensional feature spaces arise.

To address these issues and improve model stability and generalization, regularized regression models 
(Ridge and Lasso) were applied.


### 1️⃣ Ridge Regression (with Cross-Validation)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np


In [ ]:
# Ridge pipeline
ridge_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Ridge())
])


In [ ]:
# Hyperparameter grid
ridge_params = {
    'model__alpha': np.logspace(-3, 3, 20)
}


In [ ]:
# GridSearchCV
ridge_cv = GridSearchCV(
    ridge_pipe,
    ridge_params,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

ridge_cv.fit(X_train, y_train)


In [ ]:
# Best model
ridge_best = ridge_cv.best_estimator_
print("Best Ridge alpha:", ridge_cv.best_params_)


In [ ]:
y_train_pred_ridge = ridge_best.predict(X_train)
y_test_pred_ridge = ridge_best.predict(X_test)

ridge_results = {
    "Train R2": r2_score(y_train, y_train_pred_ridge),
    "Test R2": r2_score(y_test, y_test_pred_ridge),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_ridge)),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_ridge)
}

ridge_results


### 2️⃣ Lasso Regression (with Cross-Validation)

In [ ]:
from sklearn.linear_model import Lasso
lasso_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', Lasso(max_iter=10000))
])
lasso_params = {
    'model__alpha': np.logspace(-4, 1, 20)
}
lasso_cv = GridSearchCV(
    lasso_pipe,
    lasso_params,
    cv=5,
    scoring='r2',
    n_jobs=-1
)

lasso_cv.fit(X_train, y_train)


In [ ]:
lasso_best = lasso_cv.best_estimator_
print("Best Lasso alpha:", lasso_cv.best_params_)


In [ ]:
y_train_pred_lasso = lasso_best.predict(X_train)
y_test_pred_lasso = lasso_best.predict(X_test)

lasso_results = {
    "Train R2": r2_score(y_train, y_train_pred_lasso),
    "Test R2": r2_score(y_test, y_test_pred_lasso),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred_lasso)),
    "Test MAE": mean_absolute_error(y_test, y_test_pred_lasso)
}

lasso_results


In [ ]:
def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)


In [ ]:
# Number of observations
n_train = X_train_encoded.shape[0]
n_test = X_test_encoded.shape[0]

# Number of predictors
p_lr = X_train_encoded.shape[1]

# Predictions
y_train_pred_lr = mlr_pipe.predict(X_train)
y_test_pred_lr = mlr_pipe.predict(X_test)

# R2
r2_train_lr = r2_score(y_train, y_train_pred_lr)
r2_test_lr = r2_score(y_test, y_test_pred_lr)

# Adjusted R2
adj_r2_train_lr = adjusted_r2(r2_train_lr, n_train, p_lr)
adj_r2_test_lr = adjusted_r2(r2_test_lr, n_test, p_lr)

adj_r2_train_lr, adj_r2_test_lr


In [ ]:
from sklearn.linear_model import Ridge

ridge_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', Ridge(alpha=1.0))
])

ridge_pipe.fit(X_train, y_train)

y_train_pred_ridge = ridge_pipe.predict(X_train)
y_test_pred_ridge = ridge_pipe.predict(X_test)

r2_train_ridge = r2_score(y_train, y_train_pred_ridge)
r2_test_ridge = r2_score(y_test, y_test_pred_ridge)

# Ridge keeps all coefficients
p_ridge = X_train_encoded.shape[1]

adj_r2_train_ridge = adjusted_r2(r2_train_ridge, n_train, p_ridge)
adj_r2_test_ridge = adjusted_r2(r2_test_ridge, n_test, p_ridge)

adj_r2_train_ridge, adj_r2_test_ridge


In [ ]:
from sklearn.linear_model import Lasso

lasso_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', Lasso(alpha=0.001, max_iter=5000))
])

lasso_pipe.fit(X_train, y_train)

y_train_pred_lasso = lasso_pipe.predict(X_train)
y_test_pred_lasso = lasso_pipe.predict(X_test)

r2_train_lasso = r2_score(y_train, y_train_pred_lasso)
r2_test_lasso = r2_score(y_test, y_test_pred_lasso)

# Count non-zero coefficients
lasso_coefs = lasso_pipe.named_steps['model'].coef_
p_lasso = np.sum(lasso_coefs != 0)

adj_r2_train_lasso = adjusted_r2(r2_train_lasso, n_train, p_lasso)
adj_r2_test_lasso = adjusted_r2(r2_test_lasso, n_test, p_lasso)

p_lasso, adj_r2_train_lasso, adj_r2_test_lasso


In [ ]:
# Lasso feature selection
# Get feature names after preprocessing
feature_names = (
    numerical_features +
    ordinal_features +
    list(preprocessor.named_transformers_['nom']
         .get_feature_names_out(nominal_features))
)

lasso_coefs = lasso_best.named_steps['model'].coef_

selected_features = pd.Series(lasso_coefs, index=feature_names)
selected_features = selected_features[selected_features != 0].sort_values(key=abs, ascending=False)

selected_features.head(10)


In [ ]:
# Residual plots
resid_mlr = y_test - mlr_pipe.predict(X_test)
resid_ridge = y_test - ridge_best.predict(X_test)
resid_lasso = y_test - lasso_best.predict(X_test)

plt.figure(figsize=(15,4))

plt.subplot(1,3,1)
plt.scatter(mlr_pipe.predict(X_test), resid_mlr)
plt.axhline(0)
plt.title("MLR Residuals")

plt.subplot(1,3,2)
plt.scatter(ridge_best.predict(X_test), resid_ridge)
plt.axhline(0)
plt.title("Ridge Residuals")

plt.subplot(1,3,3)
plt.scatter(lasso_best.predict(X_test), resid_lasso)
plt.axhline(0)
plt.title("Lasso Residuals")

plt.tight_layout()
plt.show()


### Assumption Diagnostics Summary

Classical linear regression assumptions such as normality, homoscedasticity, 
and absence of multicollinearity were formally tested and found to be violated.

Given the large sample size and high-dimensional feature space created by 
one-hot encoding, such violations are expected in practical datasets.

To address these issues, regularized regression models (Ridge and Lasso) were employed.
While these models do not eliminate assumption violations, they mitigate their effects 
by shrinking coefficients, reducing variance, and improving generalization.

Model selection was therefore based on predictive performance rather than strict 
assumption satisfaction.


In [ ]:
import pandas as pd

results = pd.DataFrame({
    'Model': ['Linear Regression', 'Ridge Regression', 'Lasso Regression'],
    'Train R²': [r2_train, r2_train_ridge, r2_train_lasso],
    'Train Adj R²': [adj_r2_train_lr, adj_r2_train_ridge, adj_r2_train_lasso],
    'Test R²': [r2_test, r2_test_ridge, r2_test_lasso],
    'Test Adj R²': [adj_r2_test_lr, adj_r2_test_ridge, adj_r2_test_lasso],
    'Test RMSE': [
        np.sqrt(mean_squared_error(y_test, y_test_pred_lr)),
        np.sqrt(mean_squared_error(y_test, y_test_pred_ridge)),
        np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))
    ],
    'Test MAE': [
        mean_absolute_error(y_test, y_test_pred_lr),
        mean_absolute_error(y_test, y_test_pred_ridge),
        mean_absolute_error(y_test, y_test_pred_lasso)
    ]
})

results



# Final conclusion

Multiple Linear Regression achieved the highest predictive performance
with a Test R² of 0.815 and the lowest RMSE and MAE. Although some
assumptions such as normality, homoscedasticity, and multicollinearity
were violated, the model demonstrated good generalization ability.

Ridge Regression reduced coefficient instability and improved robustness
by addressing multicollinearity, but this came at a slight cost in
predictive accuracy. Lasso Regression further simplified the model by
shrinking some coefficients to zero, but resulted in the weakest
performance among the three models.

Given the objective of prediction rather than inference, Multiple Linear
Regression was selected as the final model, while Ridge Regression is a
strong alternative when model stability and interpretability are
prioritized.
